![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Finance/2.Embeddings.ipynb)

# Finance Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we got token embeddings using Spark NLP Finance Word Embeddings(**bert_embeddings_sec_bert_base**) and using these token embeddings we got sentence embeddings by sparknlp annotator SentenceEmbeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Import Required Library

In [ ]:
# Installing plotly
! pip install plotly

# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==4.0.0


## Start Spark Session

In [ ]:
import json
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

from pyspark import keyword_only
from pyspark.ml import Pipeline
from pyspark.ml import Transformer
from pyspark.ml.feature import PCA
from pyspark.ml.functions import vector_to_array
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.sql import Window
from pyspark.sql import types as T

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print ("Spark NLP Version :", sparknlp.version())

spark = sparknlp.start()

spark

Spark NLP Version : 4.0.0


# Get sample text

In [ ]:
# Downloading sample datasets.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Finance/data/finance_pca_samples.csv

In [ ]:
df = pd.read_csv("finance_pca_samples.csv")

In [ ]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+----------------+
|                text|           label|
+--------------------+----------------+
|I called Huntingt...|        Accounts|
|I opened an citi ...|        Accounts|
|I have been a lon...|    Credit Cards|
|My credit limit w...|    Credit Cards|
|I am filing this ...|Credit Reporting|
|The Credit Bureau...|Credit Reporting|
|I noticed an arti...| Debt Collection|
|A bank account wa...| Debt Collection|
|I was contacted v...|           Loans|
|My husband recent...|           Loans|
|I wire transfered...| Money Transfers|
|PayPal holds fund...| Money Transfers|
|We have requested...|        Mortgage|
|I filled out a co...|        Mortgage|
+--------------------+----------------+



# Pipeline with Spark NLP and Spark MLLIB

In [ ]:
# We defined a generic pipeline for word and sentence embeddings

def generic_pipeline():
  document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  tokenizer = Tokenizer()\
      .setInputCols("document")\
      .setOutputCol("token")

  word_embeddings = BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
      .setInputCols(["document", "token"])\
      .setOutputCol("word_embeddings")

  pipeline = Pipeline(stages = [
      document_assembler,
      tokenizer,
      word_embeddings
  ])

  return pipeline



## Sentence Embeddings

In [ ]:
embeddings_sentence = SentenceEmbeddings()\
    .setInputCols(["document", "word_embeddings"])\
    .setOutputCol("sentence_embeddings")\
    .setPoolingStrategy("AVERAGE")
# We used sparknlp SentenceEmbeddings anootator to get each sentence embeddings from token embeddings

# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [ ]:
# This class extracts the embeddings from the Spark NLP Annotation object
class EmbeddingsUDF(
    Transformer, HasInputCol, HasOutputCol,
    DefaultParamsReadable, DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: Vectors.dense(vs), VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("sentence_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = EmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
pca = pyspark.ml.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We did all process in one pipeline
pipeline = Pipeline().setStages([generic_pipeline(), embeddings_sentence, embeddings_for_pca, pca])

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]


In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result.select('pca_features', 'label').show(truncate=False)

+--------------------------------------------------------------+----------------+
|pca_features                                                  |label           |
+--------------------------------------------------------------+----------------+
|[3.395766503060028,-1.0603627660799957,-1.5687926035355224]   |Accounts        |
|[2.3660879256962364,0.8591939157072356,-0.8066160065217075]   |Accounts        |
|[0.6867755242066031,1.4823952348657887,0.00659347590660861]   |Credit Cards    |
|[-0.28833949626868444,1.0031545461252716,-0.7963772156325524] |Credit Cards    |
|[-0.5037780043184451,-1.3771577732968643,0.4449742283848416]  |Credit Reporting|
|[1.039759231614626,-1.7194163703349983,1.8539397995641318]    |Credit Reporting|
|[2.7731725080459433,1.1680247576296168,1.3949450231450957]    |Debt Collection |
|[-0.4595069206464354,0.8339685731704531,0.5051715356279516]   |Debt Collection |
|[0.27031089266955766,1.106941385143897,-0.42475361940570316]  |Loans           |
|[0.866254816450

In [ ]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

,pca_features,label
0,"[3.395766503060028, -1.0603627660799957, -1.56...",Accounts
1,"[2.3660879256962364, 0.8591939157072356, -0.80...",Accounts
2,"[0.6867755242066031, 1.4823952348657887, 0.006...",Credit Cards
3,"[-0.28833949626868444, 1.0031545461252716, -0....",Credit Cards
4,"[-0.5037780043184451, -1.3771577732968643, 0.4...",Credit Reporting
5,"[1.039759231614626, -1.7194163703349983, 1.853...",Credit Reporting
6,"[2.7731725080459433, 1.1680247576296168, 1.394...",Debt Collection
7,"[-0.4595069206464354, 0.8339685731704531, 0.50...",Debt Collection
8,"[0.27031089266955766, 1.106941385143897, -0.42...",Loans
9,"[0.8662548164505043, 1.1435248866274266, 0.870...",Loans


In [ ]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

,x,y,z,label
0,3.395767,-1.060363,-1.568793,Accounts
1,2.366088,0.859194,-0.806616,Accounts
2,0.686776,1.482395,0.006593,Credit Cards
3,-0.288339,1.003155,-0.796377,Credit Cards
4,-0.503778,-1.377158,0.444974,Credit Reporting
5,1.039759,-1.719416,1.853940,Credit Reporting
6,2.773173,1.168025,1.394945,Debt Collection
7,-0.459507,0.833969,0.505172,Debt Collection
8,0.270311,1.106941,-0.424754,Loans
9,0.866255,1.143525,0.870356,Loans


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'label', width=800, height=600)

fig.show()

### Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before. Firstly we splitted the pipeline in two to get all token embeddings

In [ ]:
model = generic_pipeline().fit(sdf)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]


In [ ]:
result = model.transform(sdf)

In [ ]:
result_df = result.select("label", F.explode(F.arrays_zip("token.result", "word_embeddings.embeddings")).alias("cols"))\
      .select(F.expr("cols['0']").alias("token"),
              "label",
              F.expr("cols['1']").alias("embeddings"))

result_df.show(truncate = 80)


+----------+--------+--------------------------------------------------------------------------------+
|     token|   label|                                                                      embeddings|
+----------+--------+--------------------------------------------------------------------------------+
|         I|Accounts|[-0.29679197, 0.80952483, 0.026026454, 0.08434165, 0.74346274, -0.02694714, -...|
|    called|Accounts|[0.28905857, -0.2922978, -0.42990327, -0.3833459, 0.026178878, -0.1272839, -0...|
|Huntington|Accounts|[0.20684443, -0.010130229, -0.25902456, -0.3755847, 0.4579211, 0.3114918, -0....|
|      Bank|Accounts|[-0.034711868, 0.460474, -0.6221116, -0.011170343, 0.29385087, 0.31341177, -0...|
|        to|Accounts|[-0.4045788, -0.37686452, -0.08015355, -0.5890965, -0.33856547, -0.39321235, ...|
|     close|Accounts|[0.35089272, 0.9568468, 0.8632823, -0.43343982, 0.11386755, -0.48837718, -0.8...|
|        my|Accounts|[-0.3659193, 0.26555997, -0.32495028, -0.5081898, -0

In [ ]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = WordEmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
pca = pyspark.ml.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We run the second part of the pipeline. Here 768 dimensions is reduced to 3 dimensions

pipeline = Pipeline().setStages([embeddings_for_pca, pca])


In [ ]:
model = pipeline.fit(result_df)

In [ ]:
result = model.transform(result_df)

In [ ]:
result.select("token", "label", "pca_features").show(truncate = 60)

+----------+--------+------------------------------------------------------------+
|     token|   label|                                                pca_features|
+----------+--------+------------------------------------------------------------+
|         I|Accounts| [9.850465344870052,0.021824808185503704,1.7128896103774132]|
|    called|Accounts| [0.5703221766039548,0.3466633993190481,-2.8677275852215174]|
|Huntington|Accounts|   [8.63544902617182,0.8802331103753407,-0.8417092198995053]|
|      Bank|Accounts| [9.391060490437583,0.45066844759817204,-1.2157440093192786]|
|        to|Accounts|[-2.0937840432194603,-1.1261841265579937,4.4733765906638725]|
|     close|Accounts|[-2.8977641355061534,-0.1632987392886953,2.6316546803841243]|
|        my|Accounts|    [3.542238995590729,-2.721487066875864,2.847899036412744]|
|   account|Accounts|[-1.2533282844778366,0.0064849964117645485,1.902319625843...|
|         ,|Accounts|  [-1.3713437013933458,0.1604400774408344,2.236148651163933]|
|   

In [ ]:
df = result.select('token', 'label', 'pca_features').toPandas()

df

,token,label,pca_features
0,I,Accounts,"[9.850465344870052, 0.021824808185503704, 1.71..."
1,called,Accounts,"[0.5703221766039548, 0.3466633993190481, -2.86..."
2,Huntington,Accounts,"[8.63544902617182, 0.8802331103753407, -0.8417..."
3,Bank,Accounts,"[9.391060490437583, 0.45066844759817204, -1.21..."
4,to,Accounts,"[-2.0937840432194603, -1.1261841265579937, 4.4..."
...,...,...,...
1364,the,Mortgage,"[0.20783234776754106, 1.2121747437322279, 2.34..."
1365,company,Mortgage,"[0.9758761373328059, 1.1525684955599134, 1.548..."
1366,never,Mortgage,"[-0.00945112197336829, -1.3605053701459042, -0..."
1367,responds,Mortgage,"[-1.310538606753127, -0.3951970841405026, -1.6..."


In [ ]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "label", "x", "y", "z"]]

df

,token,label,x,y,z
0,I,Accounts,9.850465,0.021825,1.712890
1,called,Accounts,0.570322,0.346663,-2.867728
2,Huntington,Accounts,8.635449,0.880233,-0.841709
3,Bank,Accounts,9.391060,0.450668,-1.215744
4,to,Accounts,-2.093784,-1.126184,4.473377
...,...,...,...,...,...
1364,the,Mortgage,0.207832,1.212175,2.345684
1365,company,Mortgage,0.975876,1.152568,1.548877
1366,never,Mortgage,-0.009451,-1.360505,-0.080953
1367,responds,Mortgage,-1.310539,-0.395197,-1.634089


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()